# 0. Imports

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from plotly.offline import init_notebook_mode
from bertopic import BERTopic
from bertopic import *
import nltk
# nltk.download()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from collections import Counter
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import requests
import string
import itertools
from time import sleep
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from plotly.offline import init_notebook_mode
from bertopic import BERTopic
from bertopic import *
from bertopic.representation import KeyBERTInspired
from bertopic.representation import ZeroShotClassification
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import community_detection, cos_sim
import random as rd
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


# 1. Clustering to retrieve sexual sequences

Get the data

In [3]:
f = open("gameofthrones.txt", "r",encoding='utf-8')
text_list = [line for line in f.readlines()]
delta = 5
text = ' '.join(text_list)

In [4]:
# Define a list of words in the lexical field of sex
sex_lexicon = ['sex', 'sexual', 'intimacy', 'passion', 'romance', 'affair', 'liaison', 'fling', 'tryst', 'encounter', 'activity', 'connection', 'episode', 'intercourse', 'coitus', 'lust', 'desire', 'pleasure', 'sensual', 'erotic', 'seduction', 'seductive', 'seduce', 'seducer', 'seductress', 'arousal', 'orgasm', 'foreplay', 'kiss', 'naked', 'nudity', 'pornography', 'prostitute', 'whore', 'brothel', 'virginity', 'impotence', 'erection', 'masturbation', 'ejaculation', 'contraception', 'abortion', 'homosexuality', 'bisexuality', 'transgender', 'queer', 'pansexual', 'pansexuality']
synonyms = ['rape', 'romance', 'passion', 'lust', 'affair', 'liaison', 'kiss', 'fuck', 'fucked', 'intercourse', 'coitus', 'breast', 'penis', 'masturbation', 'nipples', 'whore']
sex_lexicon.extend(synonyms)
f = open("gameofthrones.txt", "r",encoding='utf-8')
text = [line.replace("\n","") for line in f.readlines()]
text.remove("")
text_Sexual = []
delta = 0
for i in range(len(text)):
    if any(x in text[i] for x in sex_lexicon) or any(x in text[i] for x in sex_lexicon):
        # text_Sexual.append(".".join(text[i-delta : i+delta]))
        text_Sexual.append(text[i])

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(text_Sexual)

In [6]:
clusters = community_detection(torch.tensor(embeddings), threshold=0.5)
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", text_Sexual[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", text_Sexual[sentence_id])


Cluster 1, #486 Elements 
	 “One day, Bran, you will be Robb’s bannerman, holding a keep of your own for your brother and your king, and justice will fall to you. When that day comes, you must take no pleasure in the task, but neither must you look away. A ruler who hides behind paid executioners soon forgets what death is.”
	 Her brother hung the gown beside the door. “Illyrio will send the slaves to bathe you. Be sure you wash off the stink of the stables. Khal Drogo has a thousand horses, tonight he looks for a different sort of mount.” He studied her critically. “You still slouch. Straighten yourself” He pushed back her shoulders with his hands. “Let them see that you have a woman’s shape now.” His fingers brushed lightly over her budding breasts and tightened on a nipple. “You will not fail me tonight. If you do, it will go hard for you. You don’t want to wake the dragon, do you?” His fingers twisted her, the pinch cruelly hard through the rough fabric of her tunic. “Do you?” he 

In [8]:
text_aggregation = []
id1 = id2 = 0
for i in range(len(text)):
    if i < id2:
      continue
    sentence = text[i]
    id1 = id2 = i
    while len([*sentence]) < 250:
        id1 -= 1
        id2 += 1
        if id2 >= len(text):
            break
        if id1 < 0:
            break
        trsent1 = text[id1]
        trsent2 = text[id2]
        sentence = "".join([trsent1, sentence, trsent2])
    text_aggregation.append(sentence)
print(len(text_aggregation))
bookEmbeding = model.encode(text_aggregation, show_progress_bar=True)

31259


Batches:   0%|          | 0/977 [00:00<?, ?it/s]

In [9]:
cluster_descriptions = {
    8: "Intimate moments",
    10: "Brothels and desire",
    15: "Cunning remarks",
    21: "Disturbing coercion",
    23: "Traditions and sex",
    27: "Suggestive tension"
}

In [10]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# bookEmbeding.to(device)
# bookEmbeding.to(device)

In [ ]:
sexualCluster = [8, 10, 15, 21, 23, 27]
# now let make a classification model
df = pd.DataFrame(columns=['text', 'cluster', "similarity"])
total = 0
for j in tqdm(range(len(bookEmbeding))):
    maxcluster = -1
    maxsimilarity = -1 
    for i, cluster in enumerate(clusters):
        similarity = cos_sim(embeddings[cluster], bookEmbeding[j])
        if (similarity.mean() > maxsimilarity):
            maxsimilarity = similarity.mean()
            maxcluster = i
    if (maxcluster+1)  in sexualCluster:
        df = pd.concat([df, pd.DataFrame([[text_aggregation[j], cluster_descriptions[maxcluster+1], maxsimilarity.item()]], columns=['text', 'cluster', "similarity"])],  ignore_index=True)
df

 39%|███▉      | 12130/31259 [01:25<01:47, 177.87it/s]

In [ ]:
df = df[df.similarity > .4]
df.reset_index(inplace = True)
print(len(df))

In [ ]:
text_Sexual = df['text'].values
tt = []
for i in range(len(text_Sexual)):
  tt.append(str(text_Sexual[i]))
for i in range(len(df)//10):
  tt.append(text_aggregation[rd.randint(0, len(text_aggregation)-1)])

In [ ]:
urls = 'https://iceandfire.fandom.com/wiki/Category:Characters?from='
alphabet = list(string.ascii_uppercase)

soups = []
characters = []

for i in range(26):
  url = urls+(alphabet[i])
  req = urllib.request.Request(url)
  with urllib.request.urlopen(req) as response:
      html = response.read()

  soup = BeautifulSoup(html, 'html.parser')
  soups.append(soup)

characters = []

for i in range(26):
  links = soups[i].find_all('a', {'class': 'category-page__member-link'})

  for link in links:
      name = link.get_text().strip()
      if name != '':
          characters.append(name)


In [ ]:
from evaluation import evaluate_bertopic
import gc
models = ['all-MiniLM-L12-v2', 'all-MiniLM-L6-v2' , 'paraphrase-albert-small-v2', 'all-mpnet-base-v2', 'all-distilroberta-v1']
for m in models:
  p = r = f = 0
  for i in range(3):
    ptr, rtr, ftr = evaluate_bertopic(m, tt, characters, text_aggregation, .7) 
    p += ptr
    r += rtr
    f += ftr
    gc.collect()
  print(m, p/3, r/3, f/3)
  print("---------------------------------------------------") 